In [2]:
from data_download_tool import download_data, summarize_s3_structure
from annotation_viewer import plot_annotations, plot_annotation_subset
from pandas_statistics import file_path_loader
from plots import *
from aws_s3_viewer import S3Client
import os
from utilities import get_folders_in_directory, summarize_local_files, clear_local_caches, clear_local_cache, apply_bbox_corrections, apply_bbox_corrections_list, recalculate_all_statistics, generate_plots, regenerate_plots, delete_large_annotations, save_torch_script_model

path = "/data/Sentinel_Datasets/LMNT02_Raw"

# View Fits File

In [5]:
from astropy.io import fits
import numpy
import matplotlib.pyplot as plt
from preprocess_functions import iqr_log

fits_path = "/data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-09/raw_fits/61726983863263641.fits"
hdu = fits.open(fits_path)
hdul = hdu[0]

for key,value in hdul.header.items():
    print(f"{key}: {value}")

print(hdul.data.dtype)
plt.imshow(iqr_log(hdul.data).transpose(1,2,0))
plt.show()


SIMPLE: True
BITPIX: 16
NAXIS: 2
NAXIS1: 2394
NAXIS2: 1595
EXTEND: True
COMMENT:   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT:   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H
BSCALE: 1
BZERO: 32768
OBSERVER: ATIK APX60K 1422350-0244
TELESCOP: RC-16
FOCALLEN: 1828.8
APTDIA: 406.0
SBUUID: {c43e0770-a662-4e09-a439-eee2a5420374}
EXPTIME: 3.0
SWCREATE: TheSkyX Version 10.5.0 Build 11617
COLORCCD: 0
DISPINCR: 1
PICTTYPE: 1
IMAGETYP: Light Frame
XORGSUBF: 0
YORGSUBF: 0
XBINNING: 4
YBINNING: 4
CCD-TEMP: -14.79
SET-TEMP: -15.0
SITELAT: -22 13 07.26
SITELONG: +245 53 50.46
LST: +13 53 21.37
BTP: 1
CENTAZ: 19.7784342858101
CENTALT: 68.7487972897328
AIRMASS: 1.07296005726761
TELEHA: -00 28 11.96
MOUNT: Paramount ME
TELTKRA: 16.0233154
TELTKDEC: 0.4317798
OBJCTRA: 14 20 16.977
OBJCTDEC: -01 59 23.53
OBJECT: 59020
INSTRUME: ASCOM Camera
XPIXSZ: 15.039999961853
YPIXSZ: 15.039999961853
DATE-OBS: 2024-07-09T11:05:25.510
LOCALTIM: 7/9/2024 11:0

# Get Folders in Directory

In [3]:
directory = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data_finalized/RME04_MixtureC_Final"
for folder in get_folders_in_directory(directory):
    print(folder)

/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data_finalized/RME04_MixtureC_Final/RME04Sat_Training_Channel_Mixture_C
/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data_finalized/RME04_MixtureC_Final/RME04Sat-2024-07-19_Channel_Mixture_C
/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data_finalized/RME04_MixtureC_Final/RME04Sat-2024-06-09_Channel_Mixture_C
/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data_finalized/RME04_MixtureC_Final/RME04Sat-2024-06-04_Channel_Mixture_C
/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data_finalized/RME04_MixtureC_Final/RME04Sat-2024-06-11_Channel_Mixture_C
/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data_finalized/RME04_MixtureC_Final/RME04Sat-2024-06-13_Channel_Mixture_C
/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data_finalized/RME04_MixtureC_Final/RME04Sat-2024-06-08_Channel_Mixture_C
/home/davidchaparro/Repos/Dataset_Compilation_and_Statist

# Summarize directory

In [8]:
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-08 281
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-29 643
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-09 455
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-10 1112
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-11 730
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-13 406
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-12 1382
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-20-blank 4133
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-08-24-blank 654
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-08-13-blank 1284
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-08-20 1209
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-08-04 535
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-11-15 672
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-11-07 1367
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-09-05-blank 824
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-10-15 1326
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-10-23 875
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-12-17 193
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-10-30 1086
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-09-13 1220
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-11-26 429
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-12-20 264
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-09-25 492
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-10-06 450
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-12-30 276
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-12-06 222
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2025-05-25 534
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2025-05-10 648
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2025-05-03 540
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2025-01-10 759
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2025-05-16 156
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2025-01-17-blank 1071
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2025-01-07 601
# /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2025-01-23 2452
# Total Samples: 29281


summarize_local_files(path)

Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-09-06 Num Samples: 609
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-08-30 Num Samples: 90
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-07-11 Num Samples: 500
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-11-14 Num Samples: 572
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-11-07 Num Samples: 1122
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-07-08 Num Samples: 775
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-09-14 Num Samples: 1859
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-08-21 Num Samples: 1068
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-10-08 Num Samples: 463
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-12-07 Num Samples: 522
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-10-30 Num Samples: 570
Path: /data/Sentinel_Datasets/LMNT02_Raw/LMNT02Sat-2024-12-30-blank Num Samples: 3157
Path: /data/Sentinel_Datasets/LMNT02_Raw/LM

In [1]:
print(281+643+455+1112+730+406+1382+1209+535)

6753


# Generate Plots

In [6]:
generate_plots(path)

file
file
Plotting Error: Centroid positions
Plotting Error: Line Streaks


# Remove Excess Empty Frames

In [12]:
#Pandas dataframes for referenece
dataset_path = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-08-29"
local_files = file_path_loader(dataset_path)

image_attributes = local_files.statistics_file.sample_attributes
annotation_attributes = local_files.statistics_file.annotation_attributes

samples_to_remove = int(.75*len(image_attributes[image_attributes['num_objects'] == 0]))

local_files.delete_files_from_sample(image_attributes[image_attributes['num_objects'] == 0].sample(samples_to_remove))
local_files.recalculate_statistics()
regenerate_plots(dataset_path)

Plotting Error: Line Streaks


# Clear Local Cache

In [5]:
clear_local_caches(path)

Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-12/annotations
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-12/images
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-08/annotations
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-08/images
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-29/annotations
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-29/images
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-09/annotations
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-09/images
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-10/annotations
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-10/images
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-11/annotations
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-11/images
Removed: /data/Sentinel_Datasets/LMNT01_Raw/LMNT01Sat-2024-07-13/annotations
Removed: /data/Sentinel_Datas

# Apply bounding box corrections

In [ ]:
apply_bbox_corrections(path)

In [3]:

S1 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-08" # 81
S2 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-09" # 53
S4 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-14" # 257
S5 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-15" # 99
S6 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-16" # 165
S9 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-31" # 832
S10 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-04" # 427
S12 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-05" # 607
S13 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-06" # 287
S14 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-07" # 536
S15 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-08" # 215
S17 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-10" # 420
S18 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-11" # 35
S19 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-12" # 430
S20 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-13" # 663
S21 = "/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-15" # 189

new_sets = [S1, S2, S4, S5, S6, S9, S10, S12, S13, S14, S15, S17, S18, S19, S20, S21]

apply_bbox_corrections_list(new_sets)


/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-08


100%|██████████| 142/142 [00:00<00:00, 244.04it/s]


New database created at /home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-08/RME04Sat-2024-05-08_Statistics.pkl


Recalculating Statistics: 100%|██████████| 142/142 [00:05<00:00, 24.39it/s]


/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-05-09


...

Recalculating Statistics: 100%|██████████| 566/566 [00:23<00:00, 23.65it/s]


/home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-15


100%|██████████| 162/162 [00:00<00:00, 214.22it/s]


New database created at /home/davidchaparro/Repos/Dataset_Compilation_and_Statistics/data/RME04Sat-2024-06-15/RME04Sat-2024-06-15_Statistics.pkl


Recalculating Statistics: 100%|██████████| 162/162 [00:07<00:00, 22.83it/s]


# Recalculate All Statistics

In [ ]:
recalculate_all_statistics(path)


Num Samples: 1843
New database created at /mnt/c/Users/david.chaparro/Documents/Repos/Dataset_Statistics/data/LMNT01Sat-2025-04-30/LMNT01Sat-2025-04-30_Statistics.pkl


Recalculating Statistics:   1%|          | 22/1843 [00:10<13:53,  2.18it/s]


KeyboardInterrupt: 

# Convert Model to TorchScript

In [2]:
model_path = "/data/Sentinel_Datasets/Finalized_datasets/LMNT01Sat_Training_Channel_Mixture_C/models/LMNT01_MixtureC/retinanet_weights_E249.pt"
output_path = "/data/Sentinel_Datasets/Best_models"
name = "LMNT01"
save_torch_script_model(model_path, output_path,name)

/home/davidchaparro/miniconda3/envs/DatasetStatistics/lib/python3.12/site-packages/torch/nn/functional.py:4624: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  * torch.tensor(scale_factors[i], dtype=torch.float32)
/home/davidchaparro/miniconda3/envs/DatasetStatistics/lib/python3.12/site-packages/torchvision/ops/boxes.py:167: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/home/davidchaparro/miniconda3/envs/DatasetStatistics/lib/python3.12/site-packages/torchvision/ops/boxes.py:169: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detac

RuntimeError: Only tensors, lists, tuples of tensors, or dictionary of tensors can be output from traced functions